In [1]:
import numpy as np

import torch

import torch.nn as nn 
from torch.nn.utils import parameters_to_vector
import torch.optim as optim
from torchinfo import summary

import models

In [3]:
def load_checkpoint(model_path, model, optimizer=None, scheduler=None, device='cpu'):
    checkpoint = torch.load(model_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    if optimizer is not None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    if scheduler is not None:
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    start_epoch = checkpoint['epoch']
    print(f"Loading Model. Trained during {start_epoch} epochs")
    return start_epoch

In [4]:
fp32_model = models.ORIGINAL_BED_CLASSIFIER(num_classes=2).to('cpu')   

In [5]:
model_folder = './experiments_original/00_bed_original/weights/'
model_path = model_folder + 'BED_classifier__best_loss.pt'

In [6]:
load_checkpoint(model_path, fp32_model)

Loading Model. Trained during 87 epochs


87

In [8]:
# MODEL PARAMETERS
n_trainable = sum(p.numel() for p in fp32_model.parameters() if p.requires_grad)
print(f'\nTrainable parameters = {n_trainable}')

n_params = parameters_to_vector(fp32_model.parameters()).numel()
print(f'Total parameters = {n_params}\n')


Trainable parameters = 282578
Total parameters = 282578



In [9]:
fp32_model.eval()
torch_input = torch.rand(1, 3, 224, 224)
onnx_program = torch.onnx.dynamo_export(fp32_model, torch_input)

/opt/conda/envs/pytorch_23/lib/python3.10/site-packages/torch/onnx/_internal/exporter.py:136: UserWarning: torch.onnx.dynamo_export only implements opset version 18 for now. If you need to use a different opset version, please register them with register_custom_op.
  warnings.warn(
/opt/conda/envs/pytorch_23/lib/python3.10/site-packages/torch/onnx/_internal/fx/passes/readability.py:53: UserWarning: Attempted to insert a get_attr Node with no underlying reference in the owning GraphModule! Call GraphModule.add_submodule to add the necessary submodule, GraphModule.add_parameter to add the necessary Parameter, or nn.Module.register_buffer to add the necessary buffer
  new_node = self.module.graph.get_attr(normalized_name)
/opt/conda/envs/pytorch_23/lib/python3.10/site-packages/torch/fx/graph.py:1460: UserWarning: Node model_bn1_running_mean target model/bn1/running_mean model/bn1/running_mean of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typi

In [10]:
onnx_program.save("bed_original_cpu_epoch=87.onnx")